In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-rag-data/data.csv


In [24]:
!pip install accelerate -U -qq

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [25]:
!pip install bitsandbytes -qq

In [26]:
!pip install chromadb -qq

In [27]:
!pip install langchain llama-index llama_hub -qq/content/drive/MyDrive/Sem 5/DL_FINAL_PROJ/data.csv


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -/


In [28]:
!pip install sentence-transformers -qq

In [29]:
!pip install langchain-community -qq

In [30]:
!pip install -U bitsandbytes

In [31]:
# Import libraries
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import LLMChain
from langchain.prompts.prompt import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
import torch
from transformers import BitsAndBytesConfig
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import textwrap
from langchain.schema import Document

In [32]:
def generate_answer(query):
    # Only load and initialize models if they aren't already initialized
    if 'tokenizer' not in globals():
        global tokenizer, model, pipe, llm, vectorsdb, context_prompt

        # Set model name and quantization config
        name = "NousResearch/Llama-2-7b-chat-hf"
        quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                                 bnb_4bit_quant_type="nf4",
                                                 bnb_4bit_use_double_quant=True,
                                                 bnb_4bit_compute_dtype=torch.bfloat16)

        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(name, cache_dir="./model/")
        model = AutoModelForCausalLM.from_pretrained(name, cache_dir="./model/",
                                                     device_map="auto",
                                                     torch_dtype=torch.float16,
                                                     quantization_config=quantization_config)

        # Setup the text generation pipeline
        pipe = pipeline("text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=512,
                        temperature=0.4,
                        top_p=0.95,
                        repetition_penalty=1.15)
        llm = HuggingFacePipeline(pipeline=pipe)

        # Define the context-based question prompt
        context_prompt = ChatPromptTemplate.from_messages([
            ('system', "Answer the question using only the context\n\nQuestion: {question}\n\nContext: {context}"),
            ('user', "{question}")
        ])

        # Load and process CSV data
        csv_path = "/kaggle/input/nlp-rag-data/data.csv"
        df = pd.read_csv(csv_path)

        # Combine relevant text fields (adjust based on dataset structure)
        text_data = df.apply(lambda row: ' '.join(map(str, row.values)), axis=1).tolist()

        # Wrap each text string in a Document object
        documents = [Document(page_content=text) for text in text_data]

        # Split data into chunks for retrieval
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
        chunks = text_splitter.split_documents(documents)

        # Create embeddings and vector store
        embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
        embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
        vectorsdb = Chroma.from_documents(chunks, embeddings, persist_directory="db")

    # Retrieve context from vector store
    context = vectorsdb.as_retriever().get_relevant_documents(query)

    # Format the question and context for the prompt
    question_input = {'context': context, 'question': query}
    response = context_prompt.format(**question_input)

    # Invoke the model and get the result
    result = llm(response)

    # Ensure we capture only the answer text starting from the last "System:" tag
    if "System:" in result:
        answer_start = result.rfind("System:") + len("System:")
        answer = result[answer_start:].strip()

        # Remove any additional tags like "Human:" after the answer
        answer = answer.split("Human:")[0].strip()

        # Remove "Context:" if it still appears in the answer
        if "Context:" in answer:
            answer = answer.split("Context:")[-1].strip()

        # Remove introductory phrases
        if answer.startswith("Based on the provided context,"):
            answer = answer[len("Based on the provided context,"):].strip()

        # Final check: filter out any remaining document or metadata content
        if "Document(" in answer:
            answer = answer.split("Document(")[0].strip()

        return answer  # Return only the cleaned answer text

    return result

In [33]:
# Example query
query = "What is the key insight from the DOJ data?"
result = generate_answer(query)
print(result)

[


In [34]:
# Example query
query = "What is the mission or vision of the Department of Justice"
result = generate_answer(query)
print(result)

The mission or vision of the Department of Justice is facilitating administration of justice that is accessible, affordable, and efficient for all citizens.


In [35]:
# Example query
query = "What does the Department of Justice do to support justice administration?"
result = generate_answer(query)
print(result)

the Department of Justice supports justice administration by facilitating the administration of justice that ensures easy access and timely delivery of justice to all citizens.


In [36]:
# Example query/content/drive/MyDrive/Sem 5/DL_FINAL_PROJ/data.csv
query = "What are the different sections or topics covered on the DOJ website?"
result = generate_answer(query)
print(result)

the different sections or topics covered on the DOJ website are:

* Press releases
* Guidelines for submission
* Digital library
* Release of state booklets highlighting dedicated state schemes, activities of DoJ, and beneficiaries


In [37]:
# Example query
query = "What resources or documents are available in the DOJ’s digital library?"
result = generate_answer(query)
print(result)

[


In [38]:
# Example query
query = "How does the DOJ support justice administration at the state level?"
result = generate_answer(query)
print(result)

The Department of Justice (DOJ) supports justice administration at the state level through various means, including providing funding and resources to state and local law enforcement agencies, offering training and technical assistance programs, and collaborating with state and local officials on criminal justice issues.


In [39]:
# Example query
query = "What are the recent updates on the guidelines provided by the DOJ?"
result = generate_answer(query)
print(result)

[


In [40]:
# Example query
query = "How does the DOJ support justice administration at the state level?"
result = generate_answer(query)
print(result)

[


In [41]:
pip install nltk


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [45]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# Define reference answers (based on your dataset)
reference_answers = [
    ["The Department of Justice (DOJ) supports justice administration at the state level through various means, including providing funding and resources to state and local law enforcement agencies, offering training and technical assistance programs, and collaborating with state officials on criminal justice issues."],
    ["the Department of Justice supports justice administration by facilitating the administration of justice through various means such as appointing, resigning, and removing officials, enforcing contracts, and providing a nodal point for related activities."],
    ["The key insight from the DOJ data is that the Department of Justice (DOJ) has a website with various pages containing information on different topics, including press releases, history, Guidelines for submission, state booklets highlighting dedicated state schemes, and about the department."],
    ["""the following resources or documents are available in the DOJ’s digital library:
* Press releases
* State booklets highlighting dedicated state schemes, activities of DoJ, and beneficiaries
* Guidelines for submission
* Digital library
"""],

]

# Queries
queries = ["How does the DOJ support justice administration at the state level?",
           "What does the Department of Justice do to support justice administration?",
           "What is the key insight from the DOJ data?",
           "What resources or documents are available in the DOJ’s digital library?"
]

# Function to generate answers using the pre-defined `generate_answer` function
generated_answers = [generate_answer(query) for query in queries]

# Tokenize the reference and generated answers
references = [[[token for token in ref.split()] for ref in refs] for refs in reference_answers]
candidates = [gen.split() for gen in generated_answers]  # Tokenize generated answers

# Calculate the corpus-level BLEU score
smooth = SmoothingFunction().method4
bleu_score = corpus_bleu(references, candidates, smoothing_function=smooth)

# Display the BLEU score
print(f"Generalized BLEU Score: {bleu_score:.4f}")

Generalized BLEU Score: 0.6571
